# MP2

## Some useful resources:
 - Szabo and Ostlund Chapter ??
 - Levine Chapter 16
 - [Crawdad programming notes](http://sirius.chem.vt.edu/wiki/doku.php?id=crawdad:programming:project4)

# MP2 algorithm
1. The starting point will be the Hartree-Fock wavefunction. 

## Imports

In [26]:
import numpy as np
import scipy.linalg as spla
import pyscf
from pyscf import gto, scf
import matplotlib.pyplot as plt
import time
%matplotlib notebook

## Specify the molecule

In [27]:
# start timer
start_time = time.time()
# define molecule
mol = pyscf.gto.M(
    atom="O 0.0000000 0.0000000 0.0000000; H 0.7569685 0.0000000 -0.5858752; H -0.7569685 0.0000000 -0.5858752",
    basis='sto-3g',
    unit = "Ang",
    verbose=0,
    symmetry=False,
    spin = 0,
    charge = 0
)
# get number of atomic orbitals
num_ao = mol.nao_nr()
# get number of electrons
num_elec_alpha, num_elec_beta = mol.nelec
num_elec = num_elec_alpha + num_elec_beta
# get nuclear repulsion energy
E_nuc = mol.energy_nuc()

## Calculate molecular integrals 


Overlap 

$$ S_{\mu\nu} = (\mu|\nu) = \int dr \phi^*_{\mu}(r) \phi_{\nu}(r) $$

Kinetic

$$ T_{\mu\nu} = (\mu\left|-\frac{\nabla}{2}\right|\nu) = \int dr \phi^*_{\mu}(r) \left(-\frac{\nabla}{2}\right) \phi_{\nu}(r) $$

Nuclear Attraction

$$ V_{\mu\nu} = (\mu|r^{-1}|\nu) = \int dr \phi^*_{\mu}(r) r^{-1} \phi_{\nu}(r) $$

Form Core Hamiltonian

$$ H = T + V $$

Two electron integrals

$$ (\mu\nu|\lambda\sigma) = \int dr_1 dr_2 \phi^*_{\mu}(r_1) \phi_{\nu}(r_1) r_{12}^{-1} \phi^*_{\lambda}(r_2) \phi_{\sigma}(r_2) $$


In [28]:
# calculate overlap integrals
S = mol.intor('cint1e_ovlp_sph')
# calculate kinetic energy integrals
T = mol.intor('cint1e_kin_sph')
# calculate nuclear attraction integrals
V = mol.intor('cint1e_nuc_sph')
# form core Hamiltonian
H = T + V
# calculate two electron integrals
eri = mol.intor('cint2e_sph', aosym='s8')
# since we are using the 8 fold symmetry of the 2 electron integrals
# the functions below will help us when accessing elements
__idx2_cache = {}


def idx2(i, j):
    if (i, j) in __idx2_cache:
        return __idx2_cache[i, j]
    elif i >= j:
        __idx2_cache[i, j] = int(i*(i+1)/2+j)
    else:
        __idx2_cache[i, j] = int(j*(j+1)/2+i)
    return __idx2_cache[i, j]


def idx4(i, j, k, l):
    return idx2(idx2(i, j), idx2(k, l))


print(np.shape(eri))

(406,)


## Perform Hartree-Fock SCF

In [29]:
# set inital density matrix to zero
D = np.zeros((num_ao, num_ao))

# 2 helper functions for printing during SCF


def print_start_iterations():
    print("{:^79}".format("{:>4}  {:>11}  {:>11}  {:>11}  {:>11}".format(
        "Iter", "Time(s)", "RMSC DM", "delta E", "E_elec")))
    print("{:^79}".format("{:>4}  {:>11}  {:>11}  {:>11}  {:>11}".format(
        "****", "*******", "*******", "*******", "******")))


def print_iteration(iteration_num, iteration_start_time, iteration_end_time, iteration_rmsc_dm, iteration_E_diff, E_elec):
    print("{:^79}".format("{:>4d}  {:>11f}  {:>.5E}  {:>.5E}  {:>11f}".format(iteration_num,
                                                                              iteration_end_time - iteration_start_time, iteration_rmsc_dm, iteration_E_diff, E_elec)))


# set stopping criteria
iteration_max = 100
convergence_E = 1e-9
convergence_DM = 1e-5
# loop variables
iteration_num = 0
E_total = 0
E_elec = 0.0
iteration_E_diff = 0.0
iteration_rmsc_dm = 0.0
converged = False
exceeded_iterations = False

print_start_iterations()
while (not converged and not exceeded_iterations):
    # store last iteration and increment counters
    iteration_start_time = time.time()
    iteration_num += 1
    E_elec_last = E_elec
    D_last = np.copy(D)
    # form G matrix
    G = np.zeros((num_ao, num_ao))
    for i in range(num_ao):
        for j in range(num_ao):
            for k in range(num_ao):
                for l in range(num_ao):
                    G[i, j] += D[k, l] * \
                        ((2.0*(eri[idx4(i, j, k, l)])) -
                         (eri[idx4(i, k, j, l)]))
    # build fock matrix
    F = H + G
    # solve the generalized eigenvalue problem
    E_orbitals, C = spla.eigh(F, S)
    # compute new density matrix
    D = np.zeros((num_ao, num_ao))
    for i in range(num_ao):
        for j in range(num_ao):
            for k in range(num_elec_alpha):
                D[i, j] += C[i, k] * C[j, k]
    # calculate electronic energy
    E_elec = np.sum(np.multiply(D, (H + F)))
    # calculate energy change of iteration
    iteration_E_diff = np.abs(E_elec - E_elec_last)
    # rms change of density matrix
    iteration_rmsc_dm = np.sqrt(np.sum((D - D_last)**2))
    iteration_end_time = time.time()
    print_iteration(iteration_num, iteration_start_time,
                    iteration_end_time, iteration_rmsc_dm, iteration_E_diff, E_elec)
    if(np.abs(iteration_E_diff) < convergence_E and iteration_rmsc_dm < convergence_DM):
        converged = True
    if(iteration_num == iteration_max):
        exceeded_iterations = True

# calculate total energy
E_total = E_elec + E_nuc

print("{:^79}".format("Total Energy : {:>11f}".format(E_total)))

           Iter      Time(s)      RMSC DM      delta E       E_elec            
           ****      *******      *******      *******       ******            
              1     0.009933  2.69561E+00  1.27367E+02  -127.366748            
              2     0.008197  1.84626E+00  4.69671E+01   -80.399634            
              3     0.008867  1.84892E-01  4.07021E+00   -84.469843            
              4     0.007889  3.65179E-02  3.36584E-01   -84.133260            
              5     0.010010  1.41819E-02  2.77766E-02   -84.161036            
              6     0.010433  5.65413E-03  2.81184E-03   -84.158224            
              7     0.010597  2.37192E-03  1.60719E-04   -84.158064            
              8     0.009220  1.00945E-03  1.58481E-04   -84.157905            
              9     0.008328  4.33408E-04  6.31723E-05   -84.157842            
             10     0.008370  1.86875E-04  2.79308E-05   -84.157814            
             11     0.007712  8.07615E-0

# Perform MP2 calculation

## Convert the two electron integrals from AO basis to the MO basis

$$(pq|rs) = \sum_\mu \sum_\nu \sum_\lambda \sum_\sigma C_\mu^p C_\nu^q
(\mu \nu|\lambda \sigma) C_\lambda^r C_\sigma^s.$$

In [55]:
eri_mo = np.zeros((num_ao, num_ao, num_ao, num_ao))
for p in range(num_ao):
    for q in range(num_ao):
        for r in range(num_ao):
            for s in range(num_ao):
                for mu in range(num_ao):
                    for nu in range(num_ao):
                        for lmda in range(num_ao):
                            for sigma in range(num_ao):
                                eri_mo[p, q, r, s] += C[mu, p]*C[nu, q]*C[lmda,r]*C[sigma, s]*eri[idx4(mu, nu, lmda, sigma)]

### Compute the MP2 Energy

In [49]:
E_corr_mp2 = 0
for i in range(num_elec_alpha):
    for j in range(num_elec_alpha):
        for a in range(num_elec_alpha, num_ao):
            for b in range(num_elec_alpha, num_ao):
                temp = eri_mo[i, a, j, b] * \
                    (2*eri_mo[i, a, j, b] - eri_mo[i, b, j, a])
                temp /= (E_orbitals[i] + E_orbitals[j] - E_orbitals[a] - E_orbitals[b])
                E_corr_mp2 += temp

In [50]:
print('E(mp2): {}'.format(E_corr_mp2))


-0.035493175032992354


In [53]:
from pyscf import mp
m = scf.RHF(mol)
print('E(HF) = %g' % m.kernel())
mp2 = mp.MP2(m)
E_corr_mp2_pyscf = mp2.kernel()[0]
print('E(MP2) = {:.9g}'.format(E_corr_mp2_pyscf))

E(HF) = -74.9629
E(MP2) = -0.0354931531


In [54]:
#comparison from pyscf
E_diff = E_corr_mp2_pyscf - E_corr_mp2
print(E_diff)


NameError: name 'E_corr_mp2' is not defined

In [ ]:
Smart algorithm:

In [ ]:
tmp = np.zeros((num_ao*(num_ao+1)/2)*(nao)